In [119]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
import networkx as nx
import pandas as pd
import numpy as np
from tqdm import tqdm
import itertools

df=pd.read_feather("../../data/raw/work_school_home_sp.feather")

In [120]:
def clusterize_relation(df, relation_name='home', x_coord='home_x', y_coord='home_y',
                            new_col_name = 'Neighbourhood', n_participants = 10, seed=13):
    data = []
    means = []
    for zone in df[relation_name].unique():
        if pd.notna(zone):
            tmp = df[df[relation_name] == zone][['id', x_coord, y_coord, relation_name]].copy()
            n_clusters = int(np.sum(len(tmp)) / n_participants)

            if n_clusters < 2:
                tmp[new_col_name] = tmp[relation_name].astype(int).astype(str) + '_'  + '0'

            else: 
                X = tmp[[x_coord, y_coord]]
                kmeans = KMeans(n_clusters=n_clusters, random_state=seed).fit(X)
                tmp[new_col_name] = kmeans.labels_
                tmp[new_col_name] = (tmp[relation_name].astype(int).astype(str) + '_' +
                                     tmp[new_col_name].astype(int).astype(str))
                    
            data.append(tmp)
                
    df = pd.concat(data)

    return df

In [121]:
def make_clusters(df, home_cluster_n=10, work_cluster_n=20, school_cluster_n=30):
    home_clusters = clusterize_relation(df,'home','home_x','home_y','home_cluster',home_cluster_n)
    work_clusters = clusterize_relation(df,'work','work_x','work_y','work_cluster',work_cluster_n)
    school_clusters = clusterize_relation(df,'school','school_x','school_y','school_cluster',school_cluster_n)

    return merge_cluster_dataframes(df, home_clusters, work_clusters, school_clusters)

def merge_cluster_dataframes(df, home_clusters, work_clusters, school_clusters):
    merged = pd.merge(df, home_clusters.drop(['home_x', 'home_y'], axis=1), 
                                                                   on=['id', 'home'], how='left')
    
    merged = pd.merge(merged, work_clusters.drop(['work_x', 'work_y'], axis=1),
                                                                       on=['id', 'work'], how='left')
    
    merged = pd.merge(merged, school_clusters.drop(['school_x', 'school_y'], axis=1), 
                                                                             on=['id', 'school'], how='left')
    
    merged.drop(['home_x', 'home_y', 'work_x', 'work_y', 'school_x', 'school_y'], axis=1, inplace=True)
    
    assert merged[merged['home'].notna()]['home_cluster'].isna().sum() == 0
    assert merged[merged['work'].notna()]['work_cluster'].isna().sum() == 0
    assert merged[merged['school'].notna()]['school_cluster'].isna().sum() == 0
    
    return merged

In [122]:
df = make_clusters(df, 10, 20, 30)

In [123]:
def add_person_to_graph(G, person):
    G.add_node(person['id'],
               work = person['work'],
               school = person['school'],
               home = person['home'],
               age = person['idade'],
               private_healthcare = person['private_healthcare'],
               home_id = person['home_id']
    )    

def add_people_to_graph(G, df):
    print('Adding People Nodes')
    df.apply(lambda x: add_person_to_graph(G, x), axis=1)
    print(25*'*')

def add_edge(G, person1, person2, edge_type, relation_cluster, edge_zone):
    G.add_edge(person1, person2, edge_type=edge_type, cluster=relation_cluster, zone=edge_zone)

def add_edges(G, df, relation, cluster, edge_type):
    print(f'Adding {edge_type} Edges')
    total_edges = 0
    for c in tqdm(df[cluster].unique()):
        if pd.notna(c):
            tmp = df[df[cluster] == c]
            assert len(np.unique(tmp[relation])) == 1
            zone = tmp[relation].iloc[0]
            if len(tmp) > 1:
                combinations = list(itertools.combinations(tmp['id'].values, 2))
                total_edges += len(combinations)
                for p1, p2 in combinations:
                    add_edge(G,p1, p2, edge_type, c, zone)
   
    print(len([True for x,y,v in G.edges.data(data=True) if v['edge_type'] == edge_type]))
    print(25*'*')
    return total_edges

In [124]:
G = nx.MultiGraph()
add_people_to_graph(G, df)
houses = add_edges(G, df, 'home', 'home_id', 'home')
neighbors = add_edges(G, df, 'home', 'home_cluster', 'neighbor')
works = add_edges(G, df, 'work',   'work_cluster', 'work')
schools = add_edges(G, df, 'school', 'school_cluster', 'school')
assert len(G.nodes()) == len(df)
assert houses == len([True for x,y,v in G.edges.data(data=True) if v['edge_type'] == 'home'])
assert neighbors == len([True for x,y,v in G.edges.data(data=True) if v['edge_type'] == 'neighbor'])
assert works == len([True for x,y,v in G.edges.data(data=True) if v['edge_type'] == 'work'])
assert schools == len([True for x,y,v in G.edges.data(data=True) if v['edge_type'] == 'school'])


Adding People Nodes


  0%|          | 70/21708 [00:00<01:02, 344.43it/s]

*************************
Adding home Edges


  1%|          | 35/5401 [00:00<00:15, 344.87it/s]

61634
*************************
Adding neighbor Edges


...

348835
*************************
Adding work Edges


  7%|▋         | 30/423 [00:00<00:01, 298.25it/s]

342525
*************************
Adding school Edges


100%|██████████| 423/423 [00:01<00:00, 344.63it/s]


251454
*************************


In [125]:
nx.write_gpickle(G, 'SP_multiGraph.gpickle')